# Question 1

In [ ]:
import numpy as np
from scipy.optimize import linprog

class PricingGame:
    def __init__(self, num_firms=3):
        self.num_firms = num_firms
        self.costs = np.random.uniform(10, 30, num_firms)  # 每家公司的生产成本
        self.max_price = 100  # 最高可能价格
        self.market_size = 1000  # 总市场规模
        
        # 价格对需求的影响系数（随机生成）
        self.price_sensitivity = np.random.uniform(0.5, 1.5, (num_firms, num_firms))
        np.fill_diagonal(self.price_sensitivity, np.random.uniform(1.5, 2.5, num_firms))
    
    def demand(self, prices):
        base_demand = self.market_size / self.num_firms
        demands = np.full(self.num_firms, base_demand)
        for i in range(self.num_firms):
            for j in range(self.num_firms):
                if i != j:
                    demands[i] += self.price_sensitivity[i, j] * (prices[j] - prices[i])
        return np.maximum(demands, 0)  # 需求不能为负
    
    def profit(self, prices):
        demands = self.demand(prices)
        return (prices - self.costs) * demands
    
    def best_response(self, other_prices, firm_index):
        def objective(x):
            prices = np.copy(other_prices)
            prices[firm_index] = x[0]
            return -self.profit(prices)[firm_index]  # 求最大值，所以取负
        
        res = linprog(
            c=[1],
            A_ub=[[-1], [1]],
            b_ub=[-self.costs[firm_index], self.max_price],
            method='highs'
        )
        return res.x[0]
    
    def find_nash_equilibrium(self, max_iterations=100, tolerance=1e-6):
        prices = np.random.uniform(self.costs, self.max_price, self.num_firms)
        for _ in range(max_iterations):
            old_prices = np.copy(prices)
            for i in range(self.num_firms):
                other_prices = np.copy(prices)
                prices[i] = self.best_response(other_prices, i)
            if np.all(np.abs(prices - old_prices) < tolerance):
                break
        return prices

# 运行模拟
game = PricingGame()
nash_prices = game.find_nash_equilibrium()

print("公司成本:")
for i, cost in enumerate(game.costs):
    print(f"公司 {i+1}: {cost:.2f}")

print("\n纳什均衡价格:")
for i, price in enumerate(nash_prices):
    print(f"公司 {i+1}: {price:.2f}")

demands = game.demand(nash_prices)
profits = game.profit(nash_prices)

print("\n均衡下的需求量:")
for i, demand in enumerate(demands):
    print(f"公司 {i+1}: {demand:.2f}")

print("\n均衡下的利润:")
for i, profit in enumerate(profits):
    print(f"公司 {i+1}: {profit:.2f}")

公司成本:
公司 1: 26.21
公司 2: 24.98
公司 3: 14.48

纳什均衡价格:
公司 1: 53.21
公司 2: 52.27
公司 3: 46.44

均衡下的需求量:
公司 1: 283.51
公司 2: 290.97
公司 3: 425.52

均衡下的利润:
公司 1: 7655.84
公司 2: 7938.73
公司 3: 13596.09

# Question 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

class AdGame:
    def __init__(self):
        self.market_size = 1000000  # 总市场规模
        self.base_shares = np.array([0.4, 0.35, 0.25])  # 初始市场份额
        self.ad_effectiveness = np.array([0.1, 0.12, 0.15])  # 广告效果系数
        self.cost_coefficient = np.array([1.2, 1.0, 0.8])  # 成本系数
        self.max_budget = 100000  # 最大广告预算

    def market_share(self, ad_spends):
        weighted_spends = self.ad_effectiveness * ad_spends
        total_effect = np.sum(weighted_spends)
        if total_effect == 0:
            return self.base_shares
        return self.base_shares + (weighted_spends / total_effect) * (1 - np.sum(self.base_shares))

    def profit(self, ad_spends):
        shares = self.market_share(ad_spends)
        revenues = shares * self.market_size
        costs = self.cost_coefficient * ad_spends
        return revenues - costs

    def best_response(self, other_spends, company_index):
        def neg_profit(x):
            spends = np.copy(other_spends)
            spends[company_index] = x[0]
            return -self.profit(spends)[company_index]

        res = minimize(neg_profit, [self.max_budget/2], bounds=[(0, self.max_budget)], method='L-BFGS-B')
        return res.x[0]

    def find_nash_equilibrium(self, max_iterations=100, tolerance=1e-6):
        spends = np.random.uniform(0, self.max_budget, 3)
        for _ in range(max_iterations):
            old_spends = np.copy(spends)
            for i in range(3):
                other_spends = np.copy(spends)
                spends[i] = self.best_response(other_spends, i)
            if np.all(np.abs(spends - old_spends) < tolerance):
                break
        return spends

    def plot_results(self, nash_spends):
        companies = ['公司A', '公司B', '公司C']
        fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 6))

        # 广告支出
        ax1.bar(companies, nash_spends)
        ax1.set_title('纳什均衡广告支出')
        ax1.set_ylabel('广告支出')

        # 市场份额
        shares = self.market_share(nash_spends)
        ax2.pie(shares, labels=companies, autopct='%1.1f%%')
        ax2.set_title('市场份额分布')

        # 利润
        profits = self.profit(nash_spends)
        ax3.bar(companies, profits)
        ax3.set_title('各公司利润')
        ax3.set_ylabel('利润')

        plt.tight_layout()
        plt.show()

# 运行模拟
game = AdGame()
nash_spends = game.find_nash_equilibrium()
shares = game.market_share(nash_spends)
profits = game.profit(nash_spends)

print("纳什均衡广告支出:")
for i, spend in enumerate(['A', 'B', 'C']):
    print(f"公司{spend}: {nash_spends[i]:.2f}")

print("\n均衡市场份额:")
for i, share in enumerate(['A', 'B', 'C']):
    print(f"公司{share}: {shares[i]*100:.2f}%")

print("\n均衡利润:")
for i, profit in enumerate(['A', 'B', 'C']):
    print(f"公司{profit}: {profits[i]:.2f}")



纳什均衡广告支出:
公司A: 47815.46
公司B: 52530.22
公司C: 59836.74

均衡市场份额:
公司A: 37.41%
公司B: 35.63%
公司C: 26.96%

均衡利润:
公司A: 316964.88
公司B: 298621.88
公司C: 210506.54